##### Mounting the Blob Storage Container by SAS Token
sas token can be taken from the container settings > shared access tokens.
Permissions should be given Read, Write, Delete ..(sp=racwdli&st) while generating sas token. Give expire date range widely so that dont get auth error.

In [ ]:
container_name = "raw"
storage_account_name = "blobbronzelayer"
sas = "?sv=2020-02-10&st=2024-06-02T16%3A.................VVHd2cKpJWExNZqis%3D"
config = "fs.azure.sas." + container_name+ "." + storage_account_name + ".blob.core.windows.net"

try:
    dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
except:
    pass
    
try:
    dbutils.fs.mount(
      source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
      mount_point = f"/mnt/{storage_account_name}/{container_name}",
      extra_configs = {config : sas})
except:
    pass

/mnt/blobbronzelayer/raw has been unmounted.


In [ ]:
%fs mounts

In [ ]:
# Writing a file to mounting point
file = 'test'
dbutils.fs.put(f'dbfs:/mnt/{storage_account_name}/{container_name}/' + '.csv' , file, overwrite = True)

Wrote 4 bytes.
Out[13]: True

In [ ]:
# Check the existing files in mounting point
dbutils.fs.ls(f'/mnt/{storage_account_name}/{container_name}')

Out[14]: [FileInfo(path='dbfs:/mnt/blobbronzelayer/raw/.csv', name='.csv', size=4, modificationTime=1717349431000)]

##### Mounting to Container with Service Principle Credentials
To make mounting by this method, Soft Delete should be disabled for storage account. Otherwise the mounting script can give error. Go to Storage Account > Data Protection

In [ ]:
storage_account_name = 'blobbronzelayer'
container_name = "raw"
client_id = '44702cb9-f.......56a'
tenant_id = 'ea3ed546-1........403d4c'
client_secret = 'h6E8Q........dGczA'


configs = {
            "fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": f"{client_id}",
            "fs.azure.account.oauth2.client.secret": f"{client_secret}",
            "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
          }


try:
    dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
except:
    pass
    
try:
    dbutils.fs.mount(
                     source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
                     mount_point = f"/mnt/{storage_account_name}/{container_name}",
                     extra_configs = configs
                    )
except:
    pass

/mnt/blobbronzelayer/raw has been unmounted.


##### Mounting Function to Container with by Secret Scope - Azure Key Vault with Azure Key - Vault
After Creating Secret Scope in databricks and Azure Key Vault created  in Azure and settings done, we can get credentials from vault

In [ ]:
# List all the scopes
dbutils.secrets.listScopes()

Out[1]: [SecretScope(name='databricks-scope')]

In [ ]:
# List the secrets in a scope
dbutils.secrets.list("databricks-scope")

Out[2]: [SecretMetadata(key='databricks-app-client-id'),
 SecretMetadata(key='databricks-app-client-secret'),
 SecretMetadata(key='databricks-app-tenant-id')]

In [ ]:
storage_account_name = 'blobbronzelayer'
client_id = dbutils.secrets.get(scope='databricks-scope', key='databricks-app-client-id')
tenant_id = dbutils.secrets.get(scope='databricks-scope', key='databricks-app-tenant-id')
client_secret = dbutils.secrets.get(scope='databricks-scope', key='databricks-app-client-secret')


configs = {
            "fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": f"{client_id}",
            "fs.azure.account.oauth2.client.secret": f"{client_secret}",
            "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
          }


def mount_adls(container_name):
    try:
        dbutils.fs.mount(
                        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
                        mount_point = f"/mnt/{storage_account_name}/{container_name}",
                        extra_configs = configs
                        )
        print('Mounted successfully')
        
    except:
        pass

In [ ]:
mount_adls('raw')

Mounted successfully
